# QLora 实战

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "6"

4bit情况下权重的范围在[-8, 7]之间，过于稀疏导致不用非常大的离群值就使得量化后的权重集中在某些数值上，无法体现出差异。如果还像8bit情况下那样用线性量化，则如果待量化的数据为均匀分布，那量化结果不会太差，反之如果像Llama2和Chatglm3那样权重呈正态分布，那只能用$分位数量化$：对于标准正态分布N(0, 1)，可以根据PDF得到16个分位区间和16个“分位数”（实际是取17个分位数的两两平均得到的），再采用分块量化的方法，将模型权重按一组64个值统一缩放到[-1, 1]之间，最接近哪个“分位数”就用哪个作为量化值对应的浮点数，然后根据“分位数”在数组中的index将浮点数转化为整数量化值。对于反量化，同样找整数模型权重在数组中作为下标对应的“分位数”，将其视为量化值对应的浮点数，然后根据之前的缩放得到实际使用的权重。

然而，缩放过程中每组权重都需要一个fp32的量化常数（32bit，平均一下就是每个值要增加额外的32/64=0.5bit），除以这个量化常数才能得到缩放后的值，同样增加了额外存储。因此对这个量化常数也要进一步量化，一组256个值线性量化为fp8（8bit，平均一下就是每个值要增加额外的8/64+32/(64×256)=0.127bit），大大节省了存储空间，称为$双重量化$！

最后又提出了$分页优化器$的思想:当显存不足时，将优化器的参数转移到CPU内存上，需要时再将其取回，防止显存峰值时OOM。

## Step1 导入相关包

In [2]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

/home/ubuntu/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [3]:
ds = Dataset.load_from_disk("../data/alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [4]:
ds[:3]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。'],
 'input': ['', '输入：4/16', ''],
 'instruction': ['保持健康的三个提示。', '解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？']}

In [5]:
# print(len("以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。"))

## Step3 数据集预处理

In [6]:
tokenizer = AutoTokenizer.from_pretrained("/data/PLM/Llama-2-7b-ms")
tokenizer

LlamaTokenizerFast(name_or_path='/data/PLM/Llama-2-7b-ms', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	32000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
}

In [7]:
tokenizer.padding_side = "right"  # 一定要设置padding_side为right，否则batch大于1时可能不收敛

In [8]:
tokenizer.pad_token_id = 2

In [9]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    # [CLS]在LLaMA分词器中会自带，表示为<s>，可以通过add_special_tokens=False去掉，因为decoder-only的模型并不需要[CLS]等开头的token来做额外的处理；这里也还没有padding
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ", add_special_tokens=False)
    response = tokenizer(example["output"], add_special_tokens=False)
    # [EOS]在LLaMA分词器中不会自带，即使add_special_tokens=True，必须手动添加，表示为</s>
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    # 果然！labels中input的部分也需要改成-100！
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eos_token_id]
    # 这种截断方式会舍弃最后的[EOS]，可以修改一下
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [10]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [11]:
print(tokenized_ds[0]["input_ids"])

[12968, 29901, 29871, 30982, 31695, 31863, 31577, 30210, 30457, 30502, 31302, 30858, 30267, 13, 13, 7900, 22137, 29901, 29871, 29871, 30651, 30557, 30392, 30982, 31695, 31863, 31577, 30210, 30457, 30502, 31302, 30858, 30383, 13, 13, 29896, 29889, 29871, 30982, 31695, 31687, 30988, 31704, 30846, 30267, 31951, 30408, 232, 132, 157, 236, 131, 133, 30948, 30210, 31687, 30988, 31894, 30846, 30214, 30847, 233, 152, 166, 233, 176, 168, 30330, 235, 186, 148, 233, 176, 168, 31391, 233, 187, 187, 233, 182, 182, 30214, 30815, 231, 194, 134, 31174, 30869, 235, 164, 131, 31624, 31863, 31577, 30214, 232, 165, 161, 232, 191, 189, 235, 133, 143, 235, 133, 140, 31074, 31180, 30214, 31666, 30417, 31931, 30909, 232, 138, 146, 31022, 30988, 30908, 30267, 13, 13, 29906, 29889, 29871, 232, 160, 138, 235, 164, 164, 236, 168, 177, 31855, 30267, 31951, 30408, 31855, 30406, 30374, 236, 181, 159, 30210, 235, 151, 175, 31854, 30330, 30716, 30801, 30330, 30753, 31112, 30834, 30503, 235, 135, 133, 235, 133, 173, 23

In [12]:
# tokenizer("abc " + tokenizer.eos_token)

In [13]:
tokenizer.decode(tokenized_ds[0]["input_ids"])

'Human: 保持健康的三个提示。\n\nAssistant:  以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。</s>'

In [14]:
# tokenizer("呀", add_special_tokens=False) # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性

In [15]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

## Step4 创建模型

In [16]:
import torch
# load_in_4bit启用4bit线性量化，bnb_4bit_quant_type="nf4"启用nf4分位数量化, bnb_4bit_use_double_quant=True再启用双重量化
model = AutoModelForCausalLM.from_pretrained("/data/PLM/Llama-2-7b-ms", low_cpu_mem_usage=True, device_map="auto", 
                                             load_in_4bit=True, bnb_4bit_compute_dtype=torch.half, # bnb_4bit_compute_dtype指定4bit权重在推理时反量化的精度
                                             bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True) 

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.08s/it]
/home/ubuntu/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/ubuntu/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization

In [17]:
for name, param in model.named_parameters():
    print(name, param.shape, param.dtype) # 注意uint8就是int4

model.embed_tokens.weight torch.Size([32000, 4096]) torch.float16
model.layers.0.self_attn.q_proj.weight torch.Size([8388608, 1]) torch.uint8
model.layers.0.self_attn.k_proj.weight torch.Size([8388608, 1]) torch.uint8
model.layers.0.self_attn.v_proj.weight torch.Size([8388608, 1]) torch.uint8
model.layers.0.self_attn.o_proj.weight torch.Size([8388608, 1]) torch.uint8
model.layers.0.mlp.gate_proj.weight torch.Size([22544384, 1]) torch.uint8
model.layers.0.mlp.up_proj.weight torch.Size([22544384, 1]) torch.uint8
model.layers.0.mlp.down_proj.weight torch.Size([22544384, 1]) torch.uint8
model.layers.0.input_layernorm.weight torch.Size([4096]) torch.float16
model.layers.0.post_attention_layernorm.weight torch.Size([4096]) torch.float16
model.layers.1.self_attn.q_proj.weight torch.Size([8388608, 1]) torch.uint8
model.layers.1.self_attn.k_proj.weight torch.Size([8388608, 1]) torch.uint8
model.layers.1.self_attn.v_proj.weight torch.Size([8388608, 1]) torch.uint8
model.layers.1.self_attn.o_proj

In [18]:
model.config

LlamaConfig {
  "_name_or_path": "/data/PLM/Llama-2-7b-ms",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "tran

## Lora

### PEFT Step1 配置文件

In [19]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(task_type=TaskType.CAUSAL_LM,)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=None, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={})

### PEFT Step2 创建模型

In [20]:
model = get_peft_model(model, config)

In [21]:
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/data/PLM/Llama-2-7b-ms', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'v_proj', 'q_proj'}, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={})

In [22]:
for name, parameter in model.named_parameters(): # 即使模型经过了量化，Lora还是fp32的！
    print(name, parameter.dtype)

base_model.model.model.embed_tokens.weight torch.float16
base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight torch.float32
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight torch.float32
base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight torch.uint8
base_model.model.model.layers.0.self_attn.k_proj.weight torch.uint8
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight torch.float32
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight torch.float32
base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight torch.uint8
base_model.model.model.layers.0.self_attn.o_proj.weight torch.uint8
base_model.model.model.layers.0.mlp.gate_proj.weight torch.uint8
base_model.model.model.layers.0.mlp.up_proj.weight torch.uint8
base_model.model.model.layers.0.mlp.down_proj.weight torch.uint8
base_model.model.model.layers.0.input_layernorm.weight torch.float16
base_model.model.model.layers.0.post_attention_

In [23]:
model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


In [24]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [25]:
# model = model.half()  # 当整个模型都是半精度时，需要将adam_epsilon调大
# torch.tensor(1e-8).half() 

## Step5 配置训练参数

In [26]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=32,
    logging_steps=10,
    num_train_epochs=1,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit" # 开启分页优化器，这里是因为要微调的Lora权重还是fp32的
)

## Step6 创建训练器

In [27]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds.select(range(6000)),
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


## Step7 模型训练

In [28]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss


KeyboardInterrupt: 

## Step8 模型推理

In [29]:
model.eval()
ipt = tokenizer("Human: {}\n{}".format("你好", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True, eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True)

/home/ubuntu/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'Human: 你好\n\nAssistant: 嗨，仔细听取。\n\n> H: 你好 A: 嗨，仔细听取。\n\nAssistant: 那你就有仔细说仔细对了。\n\nH: 我要说迄今为止，你就不要用的那句话了。\n\nA: 你要说仔细。不然我还要说'

In [30]:
model.merge_and_unload() # 这里居然能够合并？！为什么16bit和8bit情况下不行？

/home/ubuntu/anaconda3/envs/py38/lib/python3.8/site-packages/peft/tuners/lora/bnb.py:213: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
